In [1]:
"""
Copyright (c) 2017-2022 Yuhei Otsubo
Released under the MIT license
http://opensource.org/licenses/mit-license.php
"""

'\nCopyright (c) 2017-2022 Yuhei Otsubo\nReleased under the MIT license\nhttp://opensource.org/licenses/mit-license.php\n'

In [2]:
#Experimental mode = 0(Ex.1) ~ 4(Ex.5)
args_ex_mode = 0
#Learning method: 0(UTL),1(DTL),2(UFT),3(DFT),4(2LF)
args_lm = 5

cuda_device = "cuda:1"

# for saving trained model
args_output_model = '20230104_01compiler'
args_input_model = ''#'20201005_01compiler_old'


#block size
op_num = 235
block_size = op_num
args_s_limit = 20000
# K-split cross-validation
args_k = 4
# num of epochs
args_epoch = 200
args_batch_size = 64
args_arch_num = [9,51,3,3+9,3,3+3][args_ex_mode]
args_max_norm = 2e-1#0.25 #勾配クリッピング
args_gn_rate = 1.0 #Layer-wised Gradient Normalization
# Automatic download of traind model when it is created (for Google Colab)
F_download = False#True
# Visualizing the output of the Weak Learner by T-SNE
F_arch_check = True
# Use multiple GPUs
F_multi_GPU = False#True


args_lr = 0.025 #Initial learning rate
args_momentum = 0.9 #Momentum
args_weight_decay = 1e-4 #Weight decay factor
args_smoothing = 0.1 #Label smoothing
args_dar = 0.15
args_dar2 = 0.5


In [3]:
ex_mode = ['Ex.1','Ex.2','Ex.3','Ex.4','Ex.5','Ex.6'][args_ex_mode]
learning_mode = ['UTL','DTL','UFT','DFT','2LF','DFT+','Simple'][args_lm]

if args_ex_mode >= 4:
    path = './dataset/compiler_old/'
else:
    path = './dataset/20200424compiler/'

f_pretrain_subnet = [True,False,True,False,False,False,False][args_lm]
f_pretrain_mainnet = [False,True,False,True,False,True,False][args_lm]

print(ex_mode,learning_mode)


Ex.1 DFT+


In [4]:
#for Google Colab
F_colab = False
if F_colab and 'Init_Flag' not in locals().keys():
  print('Initialization')
  #mount google drive
  from google.colab import drive
  from google.colab import files
  drive.mount('/content/drive')
  #download dataset
  !cp 'drive/My Drive/00_dataset/compiler/20200424compiler.zip' './dataset.zip'
  #!cp 'drive/My Drive/00_dataset/compiler/20200608compiler_old.zip' './dataset.zip'
  !unzip -o -qq dataset.zip

  Init_Flag = True
else:
  print('Initialization done')


Initialization done


In [5]:
import os
import random
import numpy as np
import math
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Subset
from timm.loss import LabelSmoothingCrossEntropy
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from torch import optim
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
#import secrets
from matplotlib.colors import Normalize
import itertools
import time

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 2020
seed_everything(SEED)

In [7]:
args_output = ''


In [8]:
def fild_all_files(directory):
    for root, dirs, files in os.walk(directory):
        yield root
        for file in files:
            path = os.path.join(root, file)
            if os.path.islink(path):
                continue
            yield path


In [9]:
def entropy(data):
    result = []
    s = len(data)
    for x in range(256):
        n = 0
        for i in data:
            if i == x:
                n+=1
        p_i = float(n)/s
        if p_i != 0:
                result.append(p_i * np.log2(p_i))
    r = 0.0
    for i in result:
        if i == i:
            #not NaN
            r += i
    return (-r)

In [10]:
print(path)
#Get file list

files_file = [f for f in fild_all_files(path) if os.path.isfile(os.path.join(f))]
files_file.sort()


./dataset/20200424compiler/


In [11]:
file_types = {}
file_types_ = []
num_of_file_types = {}
num_of_types = 0
for f in files_file:
  #File type classification by directory name
  file_type = f.replace(path,"").replace(os.path.basename(f),"").split("/",1)[0]
  if file_type in file_types:
    num_of_file_types[file_type] += 1
  else:
    file_types[file_type]=num_of_types
    file_types_.append(file_type)
    num_of_file_types[file_type] = 1
    print(num_of_types,file_type)
    num_of_types+=1


0 01_1_VC2003_32bit_none
1 01_2_VC2017_32bit_none
2 02_1_VC2003_32bit_max
3 02_2_VC2017_32bit_max
4 03_1_gcc6.3.0_x86_none
5 03_2_x86-O0-gcc7.5.0
6 04_1_gcc6.3.0_x86_O3
7 04_2_x86-O3-gcc7.5.0
8 05_1_clang5.0.2_32_none
9 05_2_x86-O0-Clang10.0.0
10 06_1_clang5.0.2_32_O3
11 06_2_x86-O3-Clang10.0.0
12 07_intel_32_none
13 08_intel_32bit_max
14 11_VC2017_64bit_none
15 12_VC2017_64bit_max
16 13_1_gcc6.3.0_64bit_none
17 13_2_x86_64-O0-gcc7.5.0
18 14_1_gcc6.3.0_64bit_max
19 14_2_x86_64-O3-gcc7.5.0
20 15_1_clang5.0.2_64bit_none
21 15_2_x86_64-O0-Clang10.0.0
22 16_1_clang5.0.2_64bit_max
23 16_2_x86_64-O3-Clang10.0.0
24 17_intel_64_none
25 18_intel_64bit_max
26 21_arm-O0-gcc
27 22_arm-O3-gcc
28 23_arm-O0-Clang
29 24_arm-O3-Clang
30 31_arm64-O0-gcc
31 32_arm64-O3-gcc
32 33_arm64-O0-Clang
33 34_arm64-O3-Clang
34 41_mips-O0-gcc
35 42_mips-O3-gcc
36 43_mips-O0-Clang
37 44_mips-O3-Clang
38 51_mips64-O0-gcc
39 52_mips64-O3-gcc
40 53_mips64-O0-Clang
41 54_mips64-O3-Clang
42 61_powerpc-O0-gcc
43 62_powerp

In [12]:
#label置き換え用
if args_ex_mode == 0:
    new_label_m = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
    new_label_s = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8]
elif args_ex_mode == 1:
    new_label_m = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8]
    new_label_s = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
elif args_ex_mode == 2:   
    new_label_m = [0,1,0,1,2,3,2,3,4,5,4,5,6,6,7,7,8,9,8,9,10,11,10,11,12,12,13,13,14,14,15,15,16,16,17,17,18,18,19,19,20,20,21,21,22,22,23,23,24,24,25]
    new_label_s = [0,0,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,2]
elif args_ex_mode == 3:
    new_label_m = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
    new_label_s0 = [0,0,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,2]
    new_label_s = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8]    
elif args_ex_mode == 4:
    new_label_m = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
    new_label_s = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2]
elif args_ex_mode == 5:
    new_label_m = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
    new_label_s0 = [0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,2]
    new_label_s = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2]    


In [13]:
#for calc entropy
num_of_byte = [[0.0 for x in range(256)] for y in range(num_of_types)]

In [14]:
def onehotlist(n):
  return [[0 if i!=j else 1 for i in range(n)] for j in range(n)]


In [15]:
#Dataset preparation
print('make dataset')
BitArray = [[int(x) for x in format(y,'08b')] for y in range(256)]
BitArray.append([-1 for x in range(8)])
num_of_dataset = {}
total_samples = 0
master_dataset_X = np.empty((num_of_types*args_s_limit,block_size*8+args_arch_num),dtype=np.int8)
master_dataset_Y = np.empty(num_of_types*args_s_limit,dtype=np.int8)
# master_dataset_b = []
# order_l = [[0 for i in range(32)] for j in range(num_of_types)]
random.shuffle(files_file)
if args_ex_mode == 3:
    I0 = onehotlist(3)
    I = onehotlist(9)
elif args_ex_mode == 5:
    I0 = onehotlist(3)
    I = onehotlist(3)
else:
    I = onehotlist(args_arch_num)
n = len(files_file)
i = 0
for f in files_file:
  i = i + 1
  if i % int(n/10) == 0:
    print(int(i/int(n/10)),'/10')
  ft = f.replace(path,"").replace(os.path.basename(f),"").split("/",1)[0]
  if ft not in num_of_dataset:
    num_of_dataset[ft] = 0
  if args_s_limit > 0 and num_of_dataset[ft] >= args_s_limit:
    continue
  ftype = np.int8(file_types[ft])
  fin = open(f,"rb")
  bdata = fin.read()
  fsize = len(bdata)
  if fsize < block_size:
    continue

  for c in range(0,fsize-block_size,block_size):
    if args_s_limit > 0 and num_of_dataset[ft] >= args_s_limit:
      break
    offset = c*1.0/fsize
    block = bdata[c:c+block_size]
    train = []
    #1 Byte to 8 bit-array
    for x in block:
      num_of_byte[ftype][x]+=1
    train = list(map(lambda x:BitArray[x],block))
    if args_ex_mode == 3:
        train.append(I0[new_label_s0[ftype]])
    elif args_ex_mode == 5:
        train.append(I0[new_label_s0[ftype]])
    train.append(I[new_label_s[ftype]])
    
    train = list(itertools.chain.from_iterable(train))
 
    train = np.asarray(train,dtype=np.int8)
    master_dataset_X[total_samples]=train
    master_dataset_Y[total_samples]=new_label_m[ftype]
    total_samples+=1
    num_of_dataset[ft]+=1


make dataset
1 /10
2 /10
3 /10
4 /10
5 /10
6 /10
7 /10
8 /10
9 /10
10 /10


In [16]:
#Display information about the dataset
total_samples = 0
total_files = 0
total_types = 0
t_r = 0.0
print ('label','File','Code','1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16')
for t in file_types_:
  print (t, end=' ')
  print (num_of_file_types[t], end=' ')
  print (num_of_dataset[t], end=' ')
  total_types+=1
  total_files+=num_of_file_types[t]
  total_samples+=num_of_dataset[t]
  #calc entropy
  r = 0.0
  for x in range(256):
    p_i = float(num_of_byte[file_types[t]][x])/(num_of_dataset[t]*block_size)
    if p_i != 0:
      r += p_i * np.log2(p_i)
  print ((-r), end=' ')
  t_r += (-r)
  print()
        
print ('total types', total_types)
print ('total files', total_files)
print ('total samples', total_samples)#,t_r/total_types)


label File Code 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
01_1_VC2003_32bit_none 1350 20000 5.497218434100005 
01_2_VC2017_32bit_none 1170 20000 5.357332913558505 
02_1_VC2003_32bit_max 1306 20000 6.051776864801621 
02_2_VC2017_32bit_max 1147 20000 5.873730771880074 
03_1_gcc6.3.0_x86_none 2111 20000 5.708839188017366 
03_2_x86-O0-gcc7.5.0 3006 20000 5.601603488119756 
04_1_gcc6.3.0_x86_O3 1844 20000 6.117739130113774 
04_2_x86-O3-gcc7.5.0 2756 20000 6.137322464651781 
05_1_clang5.0.2_32_none 1205 20000 5.776051052833051 
05_2_x86-O0-Clang10.0.0 3258 20000 5.473631820242736 
06_1_clang5.0.2_32_O3 1196 20000 6.2622558422432775 
06_2_x86-O3-Clang10.0.0 3278 20000 5.9776225651704795 
07_intel_32_none 1761 20000 5.225804946995282 
08_intel_32bit_max 1724 20000 6.112311167284559 
11_VC2017_64bit_none 1456 20000 4.858022796891719 
12_VC2017_64bit_max 1242 20000 5.920874953768457 
13_1_gcc6.3.0_64bit_none 1582 20000 5.3858367234951485 
13_2_x86_64-O0-gcc7.5.0 3058 20000 5.308941514545626 
14_1_g

In [17]:
# master_dataset_X =torch.FloatTensor(master_dataset_X[:total_samples])
master_dataset_Y =torch.LongTensor(master_dataset_Y[:total_samples])
# master_dataset = TensorDataset(master_dataset_X, master_dataset_Y)
# print(master_dataset_X[0],master_dataset_Y[0])

In [18]:
# PyTorch version

print(torch.__version__)

1.11.0+cu102


In [19]:
torch.cuda.is_available()

True

In [20]:
#Dataset Class
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        super().__init__()
        
        self.data = data
        self.labels = torch.LongTensor(labels)
        
        self.len = len(labels)
        self.idxs = [x for x in range(self.len)]
        random.shuffle(self.idxs)
        
    def __len__(self):
        return self.len
    
    def set_mask(mask):
        self.mask = mask
    
    def __getitem__(self, idx):
        #idx = self.idxs[idx]
        #block = copy.copy(self.data[idx])
        #block = list(map(int,self.data[idx]))
        out_label = self.labels[idx]
        real_data = np.copy(self.data[idx])
        

        return real_data, out_label
# master_dataset = MyDataset(master_dataset_X, master_dataset_Y,True)
master_dataset = MyDataset(master_dataset_X, master_dataset_Y)
x,y = master_dataset[0]
print(y,x,x.shape)
        

tensor(37) [0 0 0 ... 0 0 0] (1889,)


In [21]:
class MySubset(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, mask=None):
        self.dataset = dataset
        self.indices = indices
        self.mask = mask
        self.mask_len = int(block_size*args_dar)

    def __getitem__(self, idx):
        real_data, label = self.dataset[self.indices[idx]]
        if self.mask:
            if random.random() < args_dar2:
                da_list = random.sample(range(block_size), self.mask_len)
                for pos in da_list:

                    rnd = random.random()
                    if rnd < 0.8:
                        real_data[pos*8:pos*8+8] = BitArray[random.randrange(256)]
                    # elif rnd < 0.9:
                    #     continue
                    else:
                        # real_data[pos*8:pos*8+8] = BitArray[256]
                        continue
            

        return torch.FloatTensor(real_data), label

    def __len__(self):
        return len(self.indices)


In [22]:
class Attention(nn.Module):
  def __init__(self, length, depth):
    super(Attention,self).__init__()
    self.l_q = nn.Conv1d(depth, depth*4, 1)
    self.l_k = nn.Conv1d(depth, depth*4, 1)
    self.l_v = nn.Conv1d(depth, depth*4, 1)
    self.l_q2 = nn.Conv1d(depth*4, depth, 1)
    self.l_k2 = nn.Conv1d(depth*4, depth, 1)
    self.l_v2 = nn.Conv1d(depth*4, depth, 1)
    self.depth = depth
    self.length = length

  def forward(self,x):
    value = self.l_v(x)
    value = F.relu(value)
    value = self.l_v2(value)
    value = F.relu(value)

    attention_weight = self.get_aw(x)

    attention_output = torch.bmm(value,attention_weight)
    net_out = attention_output
        
    return net_out
  def get_aw(self, x):
    Memory = x
    query = self.l_q(x)
    key = self.l_k(x)

    query = F.relu(query)
    key = F.relu(key)

    query = self.l_q2(query)
    key = self.l_k2(key)

    logit = torch.bmm(key.permute(0,2,1),query)
    attention_weight = F.softmax(logit,dim=2)#.permute(0,2,1)

    return attention_weight

In [23]:
class PositionalEncoding(nn.Module):

  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super(PositionalEncoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)

    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0).transpose(1, 2)
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + self.pe[:,:,:x.size(2)]
    return self.dropout(x)

In [24]:
class o_glassesX(nn.Module):
    def __init__(self, depth, length, n_out, input_depth = None):
        super(o_glassesX,self).__init__()
        if input_depth == None:
            input_depth = depth
        self.linear = nn.Conv1d(input_depth, depth, 1, 1)
        self.bn = nn.BatchNorm1d(depth)
        self.pe = PositionalEncoding(depth, 0.0)
        self.att = Attention(length, depth)
        self.fc = nn.Linear(depth*length, n_out)
        
    def forward(self, x):
        mask = x > -10
        mask = torch.all(mask,dim=1,keepdim=True)
        # print(mask.shape)
        # print(b)
        #Embedding
        x = self.linear(x)
        x = F.relu(x)
        x = self.bn(x)
        x = x * mask[:,:x.shape[1],:]
        # Positional Encoding
        x = self.pe(x)
        # Attention
        x = self.att(x)
        # x = x * mask[:,:x.shape[1],:]
        
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x
    
        

In [25]:
class o_glassesX2(nn.Module):
    def __init__(self, depth, length, n_out, n_out2,input_depth):
        super(o_glassesX2,self).__init__()
        self.sub1 = o_glassesX(depth, length, n_out, input_depth)
        self.sub2 = o_glassesX(depth, length, n_out2, input_depth)

    def forward(self, x):
        x1 = self.sub1(x)
        x1 = F.softmax(x1,dim=1)
        x2 = self.sub2(x)
        x2 = F.softmax(x2,dim=1)
        
        x = torch.cat([x1,x2], dim=1)
        return x
        

In [26]:
class Model(nn.Module):
  def __init__(self, op_len, n_out):
    super(Model,self).__init__()
    self.depth = 32
    self.com_len = 4
    self.arch_num = args_arch_num
    op_len = op_len - self.com_len + 1
    if args_ex_mode == 3:
        self.sub = o_glassesX2(self.depth, op_len, 3, 9, self.depth)
    elif args_ex_mode == 5:
        self.sub = o_glassesX2(self.depth, op_len, 3, 3, self.depth)
    else:
        self.sub = o_glassesX(self.depth, op_len, self.arch_num, self.depth)
    self.main = o_glassesX(self.depth, op_len, n_out, self.depth + self.arch_num)

  @torch.no_grad()
  def im2col(self,x):
    x = torch.split(x,8,dim=1)
    x = torch.stack(x,dim=2)
    x = torch.cat([x[:,:,0:-3],x[:,:,1:-2],x[:,:,2:-1],x[:,:,3:]],dim=1)
    return x
    
  def forward(self, x, f_arch = True):

    a = x[:,-self.arch_num:]
    x = x[:,:-self.arch_num]

    # im2col
    x = self.im2col(x)
    if f_arch:
        a = self.get_arch(x)
        if args_ex_mode != 3:
            a = F.softmax(a,dim=1)

    # broadcast concatenate
    a = a[:,:,np.newaxis]
    a = a.expand(x.size(0),self.arch_num,x.size(2))
    x = torch.cat([x,a], dim=1)
        
    x = self.main(x)

    return x


  def get_arch(self,x):
    x = self.sub(x)
    # x = F.softmax(x,dim=1)
    
    return x

  def forward2(self, x):
    a = x[:,-self.arch_num:]
    x = x[:,:-self.arch_num]

    # im2col
    x = self.im2col(x)
    a = self.get_arch(x)
    if args_ex_mode != 3:
        a_ = F.softmax(a,dim=1)
    else:
        a_ = a.clone()

    # broadcast concatenate
    a_ = a_[:,:,np.newaxis]
    a_ = a_.expand(x.size(0),self.arch_num,x.size(2))
    x = torch.cat([x,a_], dim=1)
        
    x = self.main(x)

    return x,a
    



In [27]:
def time2str(elapsed_time):
    elapsed_time = int(elapsed_time)
    
    hour = elapsed_time // 3600
    minute = (elapsed_time % 3600) // 60
    second = (elapsed_time % 3600 % 60)
    
    return str(hour).zfill(2) + ":" + str(minute).zfill(2) + ":" + str(second).zfill(2)

time2str(100.1)

'00:01:40'

In [28]:
def save_model(model,file_name = args_output_model):
    # 保存
    if torch.cuda.is_available():
        model = model.to('cpu')
    torch.save(model.state_dict(),file_name+'.pth')
    model.to(device)
    #if F_download:
    #    files.download(file_name+'.pth')


In [29]:
def load_model(file_name=args_input_model):
    # 3. ネットワーク構築
    # model = nn.Sequential()
    # model.add_module('l1', Model(block_size, num_of_types))
    #model = Model(block_size, num_of_types)
    # 学習済みモデル読み込み
    model.load_state_dict(torch.load(file_name + '.pth'))
    model.to(device)
    
    return model


In [30]:
from math import log10 , floor
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

In [31]:
def clip_grad_norm_(main,sub, rate: float, norm_type: float = 2.0) -> torch.Tensor:
    rate = float(rate)
    m_params = main.parameters()
    s_params = sub.parameters()
    if isinstance(m_params, torch.Tensor):
        m_params = [m_params]
    if isinstance(s_params, torch.Tensor):
        s_params = [s_params]
    m_params = [p for p in m_params if p.grad is not None]
    s_params = [p for p in s_params if p.grad is not None]
    norm_type = float(norm_type)

    device = m_params[0].grad.device

    total_norm_s = torch.norm(torch.stack([torch.norm(p.grad.detach(), norm_type).to(device) for p in s_params]), norm_type)
    total_norm_m = torch.norm(torch.stack([torch.norm(p.grad.detach(), norm_type).to(device) for p in m_params]), norm_type)

    clip_coef = total_norm_m / max([total_norm_s,1e-16])
    if clip_coef > rate:
        clip_coef_clamped = rate * total_norm_s / max([total_norm_m,1e-16])
        for p in m_params:
            p.grad.detach().mul_(clip_coef_clamped.to(p.grad.device))
    return clip_coef


In [32]:
def GradNorm(parameters, rate: float = 1.0, norm_type: float = 2.0):
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    parameters = [p for p in parameters if p.grad is not None]
    rate = float(rate)
    norm_type = float(norm_type)
    device = parameters[0].grad.device
    
    rate = min([torch.mean(torch.stack([torch.norm(p.grad.detach(), norm_type).to(device) for p in parameters])),rate])
        
    for p in parameters:
        total_norm = torch.norm(p.grad.detach(), norm_type).to(device)
        clip_coef = rate / (total_norm  + 1e-16)
        p.grad.detach().mul_(clip_coef.to(p.grad.device))
    

In [33]:
#ラベル置き換え用
num_of_types = [51,9,26,51,19,19][args_ex_mode]
file_types_=[file_types_[t] for t in range(num_of_types)]


In [34]:
def kfold_test(k):
  global model, device
  global args_lr
  kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=0)
  mtp = [0 for j in range(num_of_types)]
  mfp = [0 for j in range(num_of_types)]
  mfn = [0 for j in range(num_of_types)]
  mtn = [0 for j in range(num_of_types)]
  mftn = [0 for j in range(num_of_types)]
  mrs = [[0 for i in range(num_of_types)] for j in range(num_of_types)]
  for fold_idx, (train_idx, test_idx) in enumerate(kf.split(master_dataset_X, master_dataset_Y)):
    print(fold_idx+1)
    arch_num = args_arch_num

    ds_train = MySubset(master_dataset, train_idx,True)
    ds_test = MySubset(master_dataset, test_idx)

    loader_train = DataLoader(ds_train, batch_size=args_batch_size, shuffle=True)
    loader_test = DataLoader(ds_test, batch_size=args_batch_size, shuffle=False)

    model = Model(op_num, num_of_types)
    model.to(device)
    
    
    # multi GPU
    if device == 'cuda' and F_multi_GPU:
        model = torch.nn.DataParallel(model)

    def main_train():
      train_loss = 0
      model.train()
      #MainNet Training
      for data, targets in loader_train:
        if torch.cuda.is_available():
          data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(data,False)
            
        loss = loss_fn(outputs, targets)
        loss.backward()
        # #Layer-wise Gradient Normalization
        # GradNorm(model.parameters(),args_gn_rate)
        optimizer.step()
        train_loss += loss.item()
      scheduler.step()
      return train_loss / len(loader_train.dataset)
        
    def sub_train():
      train_loss = 0
      model.train()
      for data, targets in loader_train:
        if torch.cuda.is_available():
          data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model.get_arch(model.im2col(data[:,:-arch_num]))
        _, targets = torch.max(data[:,-arch_num:], 1)
            
        loss = loss_fn(outputs, targets)
        loss.backward()
        # #Layer-wise Gradient Normalization
        # GradNorm(model.parameters(),args_gn_rate)
        optimizer.step()
        train_loss += loss.item()
      scheduler.step()
      return train_loss / len(loader_train.dataset)
        
    def train():
      train_loss = 0
      model.train()
      for data, targets in loader_train:
        if torch.cuda.is_available():
          data, targets = data.to(device), targets.to(device)
        if False:#for LTL
          #MainNet Training
          for p in model.main.parameters():
              p.requires_grad = True
          for p in model.sub.parameters():
              p.requires_grad = False
          optimizer.zero_grad()
          outputs = model(data,False)
            
          loss = loss_fn(outputs, targets)
          loss.backward()
          # #Layer-wise Gradient Normalization
          # GradNorm(model.parameters(),args_gn_rate)
          optimizer.step()
          #Leaky Transfer Learning
          for p in model.main.parameters():
              p.requires_grad = True
          for p in model.sub.parameters():
              p.requires_grad = True
          optimizer.zero_grad()
          outputs = model(data,True)
          loss = loss_fn(outputs, targets)
          loss.backward()
          # GradNorm(model.parameters(),args_gn_rate)
          #勾配クリッピング
          clip_grad_norm_(model.main,model.sub, args_max_norm)
        elif args_lm == 4:
          #Embedded in loss functions
          optimizer.zero_grad()
          outputs, outputs2 = model.forward2(data)
          _, targets2 = torch.max(data[:,-arch_num:], 1)
            
          loss = loss_fn(outputs, targets) + loss_fn(outputs2, targets2)
          loss.backward()
        elif args_lm < 4:#for TL,FT
          optimizer.zero_grad()
          outputs = model(data,True)
            
          loss = loss_fn(outputs, targets)
          loss.backward()
          #勾配クリッピング
          # clip_grad_norm_(model.main,model.sub, args_max_norm)
        elif args_lm ==5:#for DFT+
          optimizer.zero_grad()
          outputs = model(data,True)
            
          loss = loss_fn(outputs, targets)
          loss.backward()
        elif args_lm ==6:#for Simple
          optimizer.zero_grad()
          outputs = model(data,True)
            
          loss = loss_fn(outputs, targets)
          loss.backward()
              
            

        # GradNorm(model.parameters(),args_gn_rate)
        optimizer.step()
        train_loss += loss.item()
      scheduler.step()
      return train_loss / len(loader_train.dataset)
    
    def test(epoch=0):
      model.eval()
      if False:
        correct = 0
        correct_a = 0
        correct_a2 = 0
        train_loss = 0
        with torch.no_grad():
          for data, targets in loader_train:
            if torch.cuda.is_available():
              data, targets = data.to(device), targets.to(device)
            outputs, outputs2 = model.forward2(data)

            loss = loss_fn(outputs, targets)
            train_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct += predicted.eq(targets.data.view_as(predicted)).sum()

            if args_ex_mode == 1:
                _, predicted = torch.max(outputs2.data, 1)
                _, targets2 = torch.max(data[:,-arch_num:], 1)
                correct_a += predicted.eq(targets2.data.view_as(predicted)).sum()
                
                predicted = list(map(lambda x:new_label_m[int(x)],predicted.cpu()))
                predicted =torch.LongTensor(predicted).to(device)
                correct_a2 += predicted.eq(targets.data.view_as(predicted)).sum()
            elif args_ex_mode == 3:
                _, predicted = torch.max(outputs2[:,:3].data, 1)
                _, targets = torch.max(data[:,-arch_num:-arch_num+3], 1)
                correct_a += predicted.eq(targets.data.view_as(predicted)).sum()
                _, predicted = torch.max(outputs2[:,3:].data, 1)
                _, targets = torch.max(data[:,-arch_num+3:], 1)
                correct_a2 += predicted.eq(targets.data.view_as(predicted)).sum()
            elif args_ex_mode == 5:
                _, predicted = torch.max(outputs2[:,:3].data, 1)
                _, targets = torch.max(data[:,-arch_num:-arch_num+3], 1)
                correct_a += predicted.eq(targets.data.view_as(predicted)).sum()
                _, predicted = torch.max(outputs2[:,3:].data, 1)
                _, targets = torch.max(data[:,-arch_num+3:], 1)
                correct_a2 += predicted.eq(targets.data.view_as(predicted)).sum()
            else:
                _, predicted = torch.max(outputs2.data, 1)
                _, targets = torch.max(data[:,-arch_num:], 1)
                correct_a += predicted.eq(targets.data.view_as(predicted)).sum()

        data_num = len(loader_train.dataset)
        print('Train Acc.：{}/{}({:.2f}%),{}/{}({:.2f}%)'.format(correct, data_num, 100. * correct /data_num, correct_a, data_num, 100. * correct_a /data_num), end=" ")
        if args_ex_mode == 1 or args_ex_mode == 3 or args_ex_mode == 5:
            print(',{}/{}({:.2f}%)'.format(correct_a2, data_num, 100. * correct_a2 /data_num), end=" ")
        print(round_it(train_loss/data_num,5),end=" ")

      correct = 0
      correct_a = 0
      correct_a2 = 0
      test_loss = 0
      with torch.no_grad():
        for data, targets in loader_test:
          if torch.cuda.is_available():
            data, targets = data.to(device), targets.to(device)

          outputs, outputs2 = model.forward2(data)

          loss = loss_fn(outputs, targets)
          test_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          correct += predicted.eq(targets.data.view_as(predicted)).sum()

          if args_ex_mode == 1:
              _, predicted = torch.max(outputs2.data, 1)
              _, targets2 = torch.max(data[:,-arch_num:], 1)
              correct_a += predicted.eq(targets2.data.view_as(predicted)).sum()
            
              predicted = list(map(lambda x:new_label_m[int(x)],predicted.cpu()))
              predicted =torch.LongTensor(predicted).to(device)
              correct_a2 += predicted.eq(targets.data.view_as(predicted)).sum()
          elif args_ex_mode == 3:
              _, predicted = torch.max(outputs2[:,:3].data, 1)
              _, targets = torch.max(data[:,-arch_num:-arch_num+3], 1)
              correct_a += predicted.eq(targets.data.view_as(predicted)).sum()
              _, predicted = torch.max(outputs2[:,3:].data, 1)
              _, targets = torch.max(data[:,-arch_num+3:], 1)
              correct_a2 += predicted.eq(targets.data.view_as(predicted)).sum()
          elif args_ex_mode == 5:
              _, predicted = torch.max(outputs2[:,:3].data, 1)
              _, targets = torch.max(data[:,-arch_num:-arch_num+3], 1)
              correct_a += predicted.eq(targets.data.view_as(predicted)).sum()
              _, predicted = torch.max(outputs2[:,3:].data, 1)
              _, targets = torch.max(data[:,-arch_num+3:], 1)
              correct_a2 += predicted.eq(targets.data.view_as(predicted)).sum()
          else:
              _, predicted = torch.max(outputs2.data, 1)
              _, targets = torch.max(data[:,-arch_num:], 1)
              correct_a += predicted.eq(targets.data.view_as(predicted)).sum()

      data_num = len(loader_test.dataset)
      print('Test Acc.：{}/{}({:.2f}%),{}/{}({:.2f}%)'.format(correct, data_num, 100. * correct /data_num, correct_a, data_num, 100. * correct_a /data_num), end=" ")
      if args_ex_mode == 1 or args_ex_mode == 3 or args_ex_mode == 5:
          print(',{}/{}({:.2f}%)'.format(correct_a2, data_num, 100. * correct_a2 /data_num), end=" ")
      print(round_it(test_loss/data_num,5),end=" ")

      return test_loss

    def test_d(epoch=0):
      model.eval()
      tp = [0 for j in range(num_of_types)]
      fp = [0 for j in range(num_of_types)]
      fn = [0 for j in range(num_of_types)]
      tn = [0 for j in range(num_of_types)]
      ftn = [0 for j in range(num_of_types)]
      rs = [[0 for j2 in range(num_of_types)] for j in range(num_of_types)]
      with torch.no_grad():
        for data, targets in loader_test:
          if torch.cuda.is_available():
            data, targets = data.to(device), targets.to(device)

          outputs = model(data)

          _, predicted = torch.max(outputs.data, 1)
          for i in range(len(targets)):
            ft = targets[i]
            result = predicted[i]
            if ft == result:
              tp[ft] += 1
              tn[result] += 1
              mtp[ft] += 1
              mtn[result] += 1
            else:
              fp[ft] += 1
              fn[result] += 1
              mfp[ft] += 1
              mfn[result] += 1
            ftn[ft] += 1
            rs[ft][result]+=1
            mftn[ft] += 1
            mrs[ft][result]+=1
      for t in file_types_:
        print(t,end=" ")
      print()
      for t in file_types_:
        print(t,end=" ")
        for j in range(num_of_types):
          print(rs[file_types[t]][j],end=" ")
        print()
      print("no label Num TP FP FN TN R P F1 Acc.")
      for t in file_types_:
        ft = file_types[t]
        print(ft,end=" ")
        print(t,end=" ")
        print(ftn[ft],end=" ")
        print(tp[ft],fp[ft],fn[ft],tn[ft],end=" ")
        if tp[ft]+fn[ft] != 0:
          r = float(tp[ft])/(tp[ft]+fn[ft])
        else:
          r = 0.0
        print(r,end=" ")
        if tp[ft]+fp[ft] != 0:
          p = float(tp[ft])/(tp[ft]+fp[ft])
        else:
          p = 0.0                        
        print(p,end=" ")
        if r+p != 0:
          f1 = 2*r*p/(r+p)
        else:
          f1 = 0.0
        print(f1,end=" ")
        acc = float(tp[ft]+tn[ft])/(tp[ft]+fp[ft]+fn[ft]+tn[ft])
        print(acc)
      sum_ftn = sum(ftn)
      sum_tp = sum(tp)
      sum_fp = sum(fp)
      sum_fn = sum(fn)
      sum_tn = sum(tn)
      print('','',sum_ftn,sum_tp,sum_fp,sum_fn,sum_tn,end=" ")
      if sum_tp+sum_fn != 0:
        r = float(sum_tp)/(sum_tp+sum_fn)
      else:
        r = 0.0
      print(r,end=" ")
      if sum_tp+sum_fp != 0:
        p = float(sum_tp)/(sum_tp+sum_fp)
      else:
        p = 0.0                        
      print(p,end=" ")
      if r+p != 0:
        f1 = 2*r*p/(r+p)
      else:
        f1 = 0.0
      print(f1,end=" ")
      acc = float(sum_tp+sum_tn)/(sum_tp+sum_fp+sum_fn+sum_tn)
      print(acc)


    model_s0 = model.sub.to('cpu').state_dict()
    model.sub.to(device)

    if f_pretrain_mainnet:#for TL2,FT2
      #PreTrain Mainnet
      # Loss Functions
      if args_smoothing:
        loss_fn = LabelSmoothingCrossEntropy(smoothing=args_smoothing)
      else:
        loss_fn = torch.nn.CrossEntropyLoss()
      # Minimizer
      optimizer = torch.optim.SGD(model.main.parameters(), lr=args_lr, momentum=args_momentum, weight_decay=args_weight_decay)
      # Learning Rate Scheduler
      scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args_epoch)

      start_time = time.time()
      print("Start MainNet PreTrain")
      for epoch in range(args_epoch):
        print('epoch:',epoch+1,end=' ')
        train_loss = main_train()
        print('train_loss:', round_it(train_loss,5), end = ' ')
        elapsed_time = time.time() - start_time
        print(time2str(elapsed_time), end=" ")
        print(time2str(elapsed_time/(epoch+1)*args_epoch-elapsed_time),"left. ")
      # main_state = model.main.state_dict()
    if f_pretrain_subnet:#for TL1,FT1
      #PreTrain Subnet
      # Loss Functions
      if args_smoothing:
        loss_fn = LabelSmoothingCrossEntropy(smoothing=args_smoothing)
      else:
        loss_fn = torch.nn.CrossEntropyLoss()
      # Minimizer
      optimizer = torch.optim.SGD(model.sub.parameters(), lr=args_lr, momentum=args_momentum, weight_decay=args_weight_decay)
      # Learning Rate Scheduler
      scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args_epoch)

      start_time = time.time()
      print("Start SubNet PreTrain")
      for epoch in range(args_epoch):
        print('epoch:',epoch+1,end=' ')
        train_loss = sub_train()
        print('train_loss:', round_it(train_loss,5), end = ' ')
        elapsed_time = time.time() - start_time
        print(time2str(elapsed_time), end=" ")
        print(time2str(elapsed_time/(epoch+1)*args_epoch-elapsed_time),"left. ")
      # sub_state = model.sub.state_dict()
        
    # if args_lm == 5 or args_lm==1:#for DFT+,DTL2
    if args_lm == 5:#for DFT+
      #warmup Subnet
      # Loss Functions
      # args_lr=0.0001
      if args_smoothing:
        loss_fn = LabelSmoothingCrossEntropy(smoothing=args_smoothing)
      else:
        loss_fn = torch.nn.CrossEntropyLoss()
      # Minimizer
      optimizer = torch.optim.SGD(model.sub.parameters(), lr=args_lr, momentum=args_momentum, weight_decay=args_weight_decay)
      # Learning Rate Scheduler
      scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args_epoch)

      start_time = time.time()
      print("Start SubNet warmup")
      for epoch in range(args_epoch):
        print('epoch:',epoch+1,end=' ')
        train_loss = train()
        print('train_loss:', round_it(train_loss,5), end = ' ')
        test_loss = test(epoch+1)
        elapsed_time = time.time() - start_time
        print(time2str(elapsed_time), end=" ")
        print(time2str(elapsed_time/(epoch+1)*args_epoch-elapsed_time),"left. ")
      # sub_state = model.sub.state_dict()
      args_lr=0.025

    # Loss Functions
    if args_smoothing:
      loss_fn = LabelSmoothingCrossEntropy(smoothing=args_smoothing)
    else:
      loss_fn = torch.nn.CrossEntropyLoss()
    # Minimizer
    if args_lm >=2:#fot LTL,FT,2loss,Simple
        optimizer = torch.optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum, weight_decay=args_weight_decay)
    elif args_lm == 0:#for TL1
        optimizer = torch.optim.SGD(model.main.parameters(), lr=args_lr, momentum=args_momentum, weight_decay=args_weight_decay)
    elif args_lm == 1:#for TL2
        optimizer = torch.optim.SGD(model.sub.parameters(), lr=args_lr, momentum=args_momentum, weight_decay=args_weight_decay)
        
    # Learning Rate Scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args_epoch)

    mini_loss = test()
    print()
    save_model(model, args_output_model)
    opt_state = optimizer.state_dict()


    start_time = time.time()
    for epoch in range(args_epoch):
      print('epoch:',epoch+1,end=' ')
      train_loss = train()
      print('train_loss:', round_it(train_loss,5), end = ' ')
      # if epoch == 0:
      #   mini_loss = train_loss
      #   save_model(model, args_output_model)
      if False:#train_loss > 1.1 * mini_loss:
        model = load_model(args_output_model)
        print('rerload model',epoch+1, end = ' ')
      else:
        test_loss = test(epoch+1)
        if mini_loss > train_loss:
          mini_loss = train_loss
          opt_state = optimizer.state_dict()
          save_model(model, args_output_model)
          print("Save model",end=' ')
        # elif mini_loss*1.1 < train_loss:
        #   # model = load_model(args_output_model)
        #   load_model(args_output_model)
        #   optimizer.load_state_dict(opt_state)
        #   print("Load model",end=' ')
      elapsed_time = time.time() - start_time
      print(time2str(elapsed_time), end=" ")
      print(time2str(elapsed_time/(epoch+1)*args_epoch-elapsed_time),"left. ")

    #subnetの貢献計算用
    # load_model(args_output_model)
    model.sub.load_state_dict(model_s0)
    model.sub.to(device)
    test(0)
    print('')
    
    model = load_model(args_output_model)

    test_d()
    if args_output_model:
        # Saving the trained model
        if torch.cuda.is_available():
            model = model.to('cpu')
        torch.save(model.state_dict(),args_output_model+'.pth')
        if F_download:
          files.download(args_output_model+'.pth')
        # return
    # return
  print('All results')
  for t in file_types_:
    print(t,end=" ")
  print()
  for t in file_types_:
    print(t,end=" ")
    for j in range(num_of_types):
      print(mrs[file_types[t]][j],end=" ")
    print()
  print("no label Num TP FP FN TN R P F1 Acc.")
  for t in file_types_:
    ft = file_types[t]
    print(ft,end=" ")
    print(t,end=" ")
    print(mftn[ft],end=" ")
    print(mtp[ft],mfp[ft],mfn[ft],mtn[ft],end=" ")
    if mtp[ft]+mfn[ft] != 0:
      r = float(mtp[ft])/(mtp[ft]+mfn[ft])
    else:
      r = 0.0
    print(r,end=" ")
    if mtp[ft]+mfp[ft] != 0:
      p = float(mtp[ft])/(mtp[ft]+mfp[ft])
    else:
      p = 0.0                        
    print(p,end=" ")
    if r+p != 0:
      f1 = 2*r*p/(r+p)
    else:
      f1 = 0.0
    print(f1,end=" ")
    acc = float(mtp[ft]+mtn[ft])/(mtp[ft]+mfp[ft]+mfn[ft]+mtn[ft])
    print(acc)
  sum_mftn = sum(mftn)
  sum_mtp = sum(mtp)
  sum_mfp = sum(mfp)
  sum_mfn = sum(mfn)
  sum_mtn = sum(mtn)
  print('','',sum_mftn,sum_mtp,sum_mfp,sum_mfn,sum_mtn,end=" ")
  if sum_mtp+sum_mfn != 0:
    r = float(sum_mtp)/(sum_mtp+sum_mfn)
  else:
    r = 0.0
  print(r,end=" ")
  if sum_mtp+sum_mfp != 0:
    p = float(sum_mtp)/(sum_mtp+sum_mfp)
  else:
    p = 0.0                        
  print(p,end=" ")
  if r+p != 0:
    f1 = 2*r*p/(r+p)
  else:
    f1 = 0.0
  print(f1,end=" ")
  acc = float(sum_mtp+sum_mtn)/(sum_mtp+sum_mfp+sum_mfn+sum_mtn)
  print(acc)


In [35]:
def detect(bb):
    x = bb
    y = model(x)
    
    _, predicted = torch.max(y,1)
    print(predicted)
    
    aw = model.get_aw(x)
    
    l2 = torch.norm(aw, 2, dim=2)
    aw2 = [0.0 for i in range(op_num)]
    for i in range(len(l2[0])):
      aw2[i]+=float(l2[0][i])
    return predicted,aw2
    

In [36]:
device = torch.device(cuda_device if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [37]:
kfold_test(args_k)

1
Start MainNet PreTrain
epoch: 1 train_loss: 0.018046 00:01:47 05:57:58 left. 
epoch: 2 train_loss: 0.015272 00:03:36 05:57:58 left. 
epoch: 3 train_loss: 0.014791 00:05:29 06:00:29 left. 
epoch: 4 train_loss: 0.014547 00:07:28 06:06:06 left. 
epoch: 5 train_loss: 0.01438 00:09:28 06:09:48 left. 
epoch: 6 train_loss: 0.014256 00:11:27 06:10:27 left. 
epoch: 7 train_loss: 0.014163 00:13:26 06:10:35 left. 
epoch: 8 train_loss: 0.014098 00:15:25 06:10:04 left. 
epoch: 9 train_loss: 0.014062 00:17:24 06:09:24 left. 
epoch: 10 train_loss: 0.014007 00:19:25 06:09:02 left. 
epoch: 11 train_loss: 0.013972 00:21:25 06:08:05 left. 
epoch: 12 train_loss: 0.013944 00:23:24 06:06:45 left. 
epoch: 13 train_loss: 0.013911 00:25:23 06:05:16 left. 
epoch: 14 train_loss: 0.013898 00:27:23 06:03:55 left. 
epoch: 15 train_loss: 0.013866 00:29:22 06:02:17 left. 
epoch: 16 train_loss: 0.013845 00:31:23 06:00:59 left. 
epoch: 17 train_loss: 0.013823 00:33:22 05:59:20 left. 
epoch: 18 train_loss: 0.013834 00

In [38]:
print(model)
for i,p in enumerate(model.parameters()):
    print(i,p.data.shape)



Model(
  (sub): o_glassesX(
    (linear): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pe): PositionalEncoding(
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (att): Attention(
      (l_q): Conv1d(32, 128, kernel_size=(1,), stride=(1,))
      (l_k): Conv1d(32, 128, kernel_size=(1,), stride=(1,))
      (l_v): Conv1d(32, 128, kernel_size=(1,), stride=(1,))
      (l_q2): Conv1d(128, 32, kernel_size=(1,), stride=(1,))
      (l_k2): Conv1d(128, 32, kernel_size=(1,), stride=(1,))
      (l_v2): Conv1d(128, 32, kernel_size=(1,), stride=(1,))
    )
    (fc): Linear(in_features=7424, out_features=9, bias=True)
  )
  (main): o_glassesX(
    (linear): Conv1d(41, 32, kernel_size=(1,), stride=(1,))
    (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pe): PositionalEncoding(
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (att): Attention(
     

In [39]:
# if args_output_model:  
#     # model = nn.Sequential()
#     # model.add_module('l1', Model(op_num, num_of_types))
#     model = Model(block_size, num_of_types)
#     model.load_state_dict(torch.load(args_output_model + '.pth'))
#     device = torch.device('cpu')
    
#     model.eval()
        
#     bb = master_dataset_X[0:1]

#     print(bb)
#     t = master_dataset_Y[0:10]
#     print(t)

    
#     p = model(master_dataset_X[0:10])
#     print(p)


In [40]:
# if F_arch_check:
#   from sklearn.manifold import TSNE
#   from sklearn.model_selection import ShuffleSplit
#   # loader_test = DataLoader(master_dataset, batch_size=args_batch_size, shuffle=False)
#   test_size = min(0.5,1000*51/len(master_dataset_X))
#   ss = ShuffleSplit(n_splits=1, test_size=test_size, random_state=0)
#   for fold_idx, (train_idx, test_idx) in enumerate(ss.split(master_dataset_X, master_dataset_Y)):
#     print(fold_idx+1)
#     arch_num = args_arch_num

#     # ds_train = Subset(master_dataset, train_idx)
#     ds_test = Subset(master_dataset, test_idx)
#     ds_test_y = master_dataset_Y[test_idx]
#     loader_test = DataLoader(ds_test, batch_size=args_batch_size, shuffle=False)
#   model.to(device)
#   model.eval()
#   arch = []
#   arch_num = args_arch_num
#   with torch.no_grad():
#     for data, targets in loader_test:
#       if torch.cuda.is_available():
#         data, targets = data.to(device), targets.to(device)
      
#       x = torch.split(data[:,:-arch_num],8,dim=1)
#       x = torch.stack(x,dim=2)
#       x = torch.cat([x[:,:,0:-3],x[:,:,1:-2],x[:,:,2:-1],x[:,:,3:]],dim=1)


#       outputs = model.get_arch(x).to('cpu').detach().numpy().copy()
#       arch.append(outputs)
#   arch = np.concatenate(arch)

#   print(arch.shape)
#   targets = ds_test_y.to('cpu').detach().numpy().copy()
#   print(targets.shape)
#   #Color Settings
#   for i in range(len(targets)):
#     t = targets[i]
#     if t <= 13:
#       #x86
#       c = 0
#     elif t <= 25:
#       #x86-64
#       c = 1
#     elif t <= 29:
#       #Arm
#       c = 2
#     elif t <= 33:
#       #Arm64
#       c = 3
#     elif t <= 37:
#       #mips
#       c = 4
#     elif t <= 41:
#       #mips64
#       c = 5
#     elif t <= 45:
#       #powerpc
#       c = 6
#     elif t <= 49:
#       #powerpc64
#       c = 7
#     else:
#       #others
#       c = 8
#     targets[i] = c
#   X_reduced = TSNE(n_components=2, random_state=0).fit_transform(arch)
#   print(X_reduced.shape)
#   plt.figure(dpi=200)
#   plt.scatter(X_reduced[:,0], X_reduced[:,1], s = 0.1, c = targets[:], cmap='jet')
#   plt.colorbar()

In [41]:
# from bokeh.plotting import figure, output_file, show, ColumnDataSource
# from bokeh.palettes import Turbo256
# from bokeh.transform import linear_cmap
# from bokeh.io import output_notebook

# # 出力設定
# output_notebook()

# mapper = linear_cmap(field_name="l", palette=Turbo256, low=min(targets), high=max(targets))
# label = ["x86","x86-64","ARM","ARM64","MIPS","MIPS64","PowerPC","PowerPC64","Others"]
# # 辞書
# source = ColumnDataSource(data=dict(
#     x=X_reduced[:,0],
#     y=X_reduced[:,1],
#     c=[label[i] for i in targets],
#     l=targets,
# ))

# # tooltips設定
# TOOLTIPS = [
#     ("index", "$index"),
#     ("x", "$x"),
#     ("y", "$y"), 
#     ("label", "@c"),
# ]


# # グラフ全体の設定
# p = figure(
#             title="test",
#             plot_width=800,
#             plot_height=600,
#             x_axis_label='x', 
#             y_axis_label='y',
#             tooltips=TOOLTIPS
#           )


# # 描画
# p.circle('x', 'y', color=mapper, fill_alpha=0.5, size=5, source=source)

# # 出力
# output_file("iris.html")
# show(p)